In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers


In [3]:
import tiktoken
import torch
from notebooks.gpt2_models.dummy_model import GPT, GPTConfig
from torch.nn import functional as F
import time

#### Model Init using Pretrained weights for GPT-2 from HF
Initiate the model with pretrained weights from HF for Gpt2 and try creating 5 predicted sequences using same instruction prompts

In [4]:
num_return_sequences = 5 # Number of sentences to generate
max_length =30 # Maximum length of the sentence

In [5]:
pretrained_model = GPT.from_pretrained("gpt2")
print(pretrained_model)

loading weights from pretrained gpt: gpt2


GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


What do we have - First and embedding layer with vocab size of 50257 and embedding size of 768. Post which we have pos embedding of 1024 block size .Then we have 12 transformer decoder layers with 12 heads each. Each decoder layer is build up with LayerNorm -> MultiHeadAttention -> LayerNorm -> FeedForward -> LayerNorm. The output of the last decoder layer is passed through a linear layer to get the logits for the next token prediction.

In [6]:
# basic settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)
torch.cuda.manual_seed(42)
device_type = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
pretrained_model.to(device)
pretrained_model.eval()

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
dummy_str = "Hello I am a language Model, I am here to help you."
encoder = tiktoken.get_encoding(encoding_name="gpt2")
tokens = encoder.encode(dummy_str)
tokens_vec = torch.tensor(tokens, dtype=torch.long)
print(tokens_vec.shape)

torch.Size([14])


In [9]:
# creating a batch of repeated tokens
tokens_batch = tokens_vec.unsqueeze(0).repeat(num_return_sequences, 1)
tokens_batch = tokens_batch.to(device)
print(tokens_batch.shape)

torch.Size([5, 14])


In [10]:
def batch_prediction(xgen, model):

    xgen = xgen.clone()

    while xgen.size(1) < max_length:

        # forward the model to get the logits
        with torch.no_grad():
            logits, loss = model(xgen)  # (B, T, vocab_size)
            # take the logits at the last position
            logits = logits[:, -1, :]  # (B, vocab_size)
            # get the probabilities
            probs = F.softmax(logits, dim=-1)
            # do top-k sampling of 50 (huggingface pipeline default)
            # topk_probs here becomes (5, 50), topk_indices is (5, 50)
            topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
            # select a token from the top-k probabilities
            # note: multinomial does not demand the input to sum to 1
            ix = torch.multinomial(topk_probs, 1)  # (B, 1)
            # gather the corresponding indices
            xcol = torch.gather(topk_indices, -1, ix)  # (B, 1)
            # append to the sequence
            xgen = torch.cat((xgen, xcol), dim=1)

    return xgen

In [11]:
prediction_batch = batch_prediction(tokens_batch, pretrained_model)

In [12]:
print(prediction_batch.shape)

torch.Size([5, 30])


In [13]:
for i in range(prediction_batch.size(0)):
    x = prediction_batch[i, :max_length].tolist()
    decoded_str = encoder.decode(x)
    print(f">>{decoded_str}")
    print("\n")

>>Hello I am a language Model, I am here to help you. I am very happy with where this world would be if you could give me something


>>Hello I am a language Model, I am here to help you.

Thank you,

Emma<|endoftext|>Liz Peltz:


>>Hello I am a language Model, I am here to help you.

But as you say, my job now is to get things done.


>>Hello I am a language Model, I am here to help you.

I am a language model, I am here to help you. You


>>Hello I am a language Model, I am here to help you. If you have no idea what a data model is:

So. what




#### Basic Model Training of the model with toy data

Lets define the data loader for the model

In [14]:
encoder = tiktoken.get_encoding(encoding_name="gpt2")
filename = "./data/tinyshakespeare/input.txt"

In [15]:
def read_tokens(filename, encoder):
    with open(filename, "r") as file:
        lines = file.read()

    tokens = encoder.encode(lines)
    return tokens

In [16]:
class DataloaderLiter:
    def __init__(self, B, T, encoder, filename):
        self.B = B  # batch size
        self.T = T  # seq length
        tokens = read_tokens(filename, encoder)
        self.tokens = torch.tensor(tokens, dtype=torch.long)

        print(f"Total tokens in the file: {len(self.tokens)}")
        print(f"1 epoch will have {len(self.tokens) // (B * T)} iterations")

        self.current_position = 0
        self.num_iterations = len(self.tokens) // (B * T)

    def next_batch(self):
        B, T = self.B, self.T
        current_batch = self.tokens[
            self.current_position : self.current_position + (B * T) + 1
        ]  # +1 for the target
        x = current_batch[:-1].view(B, T)  # input tensor of shape (B, T)
        y = current_batch[1:].view(B, T)  # target tensor of shape (B, T)

        # updating the position for next batch
        self.current_position += B * T

        # we reset the position if the next batch is OOB
        if self.current_position + (B * T) + 1 > len(self.tokens):
            self.current_position = 0

        return x, y

In [17]:
train_loader = DataloaderLiter(B=8, T=1024, encoder=encoder, filename=filename)

Total tokens in the file: 338025
1 epoch will have 41 iterations


In [18]:
num_epochs = 2
initial_iterations = 100
total_iteration = num_epochs * train_loader.num_iterations
print(f"Total iterations: {total_iteration}")

Total iterations: 82


In [19]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

In [20]:
transformer_model = GPT(GPTConfig())
transformer_model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [21]:
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=3e-4)

In [22]:
transformer_model.train()
for iter in range(initial_iterations):
    start_time = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    logits, loss = transformer_model(x, y)
    loss.backward()
    optimizer.step()
    end_time = time.time()
    torch.cuda.empty_cache()  # clear the cache
    torch.cuda.synchronize()  # wait for the computation to be done
    time_elapsed = (end_time - start_time) * 1000
    token_persec = x.numel() / time_elapsed
    print(
        f"for iter: {iter}, loss: {loss.item()}, time: {time_elapsed:.2f}ms, tokens/mssec: {token_persec:.2f}"
    )

for iter: 0, loss: 11.010125160217285, time: 1204.90ms, tokens/mssec: 6.80
for iter: 1, loss: 9.599913597106934, time: 219.55ms, tokens/mssec: 37.31
for iter: 2, loss: 8.829021453857422, time: 249.11ms, tokens/mssec: 32.88
for iter: 3, loss: 8.613097190856934, time: 256.11ms, tokens/mssec: 31.99
for iter: 4, loss: 8.430353164672852, time: 253.04ms, tokens/mssec: 32.37
for iter: 5, loss: 8.372970581054688, time: 234.27ms, tokens/mssec: 34.97
for iter: 6, loss: 8.321300506591797, time: 249.31ms, tokens/mssec: 32.86
for iter: 7, loss: 7.983493804931641, time: 253.12ms, tokens/mssec: 32.36
for iter: 8, loss: 7.712906837463379, time: 250.26ms, tokens/mssec: 32.73
for iter: 9, loss: 7.669942378997803, time: 245.65ms, tokens/mssec: 33.35
for iter: 10, loss: 7.633736610412598, time: 257.81ms, tokens/mssec: 31.77
for iter: 11, loss: 7.45471715927124, time: 246.34ms, tokens/mssec: 33.25
for iter: 12, loss: 7.39841890335083, time: 251.71ms, tokens/mssec: 32.54
for iter: 13, loss: 7.17097425460815

Precision settings - Application of tf float32 to increase throughput

In [25]:
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn
torch.backends.cudnn.benchmark = True # set to true for faster training
torch.set_float32_matmul_precision('high')

In [26]:
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()
torch.cuda.synchronize()

In [27]:
transformer_model = GPT(GPTConfig())
transformer_model.to(device)

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (h): ModuleList(
      (0-11): 12 x Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (c_attn): Linear(in_features=768, out_features=2304, bias=True)
          (c_proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Linear(in_features=768, out_features=3072, bias=True)
          (gelu): GELU(approximate='tanh')
          (c_proj): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [28]:
optimizer = torch.optim.Adam(transformer_model.parameters(), lr=3e-4)

In [29]:
transformer_model.train()
for iter in range(initial_iterations):
    start_time = time.time()
    x, y = train_loader.next_batch()
    x, y = x.to(device), y.to(device)
    optimizer.zero_grad()
    logits, loss = transformer_model(x, y)
    loss.backward()
    optimizer.step()
    end_time = time.time()
    torch.cuda.empty_cache()  # clear the cache
    torch.cuda.synchronize()  # wait for the computation to be done
    time_elapsed = (end_time - start_time) * 1000
    token_persec = x.numel() / time_elapsed
    print(
        f"for iter: {iter}, loss: {loss.item()}, time: {time_elapsed:.2f}ms, tokens/mssec: {token_persec:.2f}"
    )

for iter: 0, loss: 10.890664100646973, time: 42.46ms, tokens/mssec: 192.94
for iter: 1, loss: 9.615824699401855, time: 240.92ms, tokens/mssec: 34.00
for iter: 2, loss: 9.006220817565918, time: 268.63ms, tokens/mssec: 30.50
for iter: 3, loss: 8.814001083374023, time: 273.68ms, tokens/mssec: 29.93
for iter: 4, loss: 8.714908599853516, time: 271.72ms, tokens/mssec: 30.15
for iter: 5, loss: 8.374526977539062, time: 274.42ms, tokens/mssec: 29.85
for iter: 6, loss: 8.152328491210938, time: 278.57ms, tokens/mssec: 29.41
for iter: 7, loss: 7.994481563568115, time: 266.88ms, tokens/mssec: 30.70
for iter: 8, loss: 7.850434303283691, time: 257.98ms, tokens/mssec: 31.75
for iter: 9, loss: 7.57218074798584, time: 270.49ms, tokens/mssec: 30.29
for iter: 10, loss: 7.56642484664917, time: 272.42ms, tokens/mssec: 30.07
for iter: 11, loss: 7.270780086517334, time: 261.13ms, tokens/mssec: 31.37
for iter: 12, loss: 7.126434803009033, time: 281.27ms, tokens/mssec: 29.13
for iter: 13, loss: 6.97861480712890